## Konvertere observasjoner registrert i  artsobeservasjoner.no over til ei fil som kan importeres til ebird.org 
Her gjør jeg jobben i Python og jupyter Notebook. Det meste kan gjøres i Excel også, men når opplegget her fungerer, så er det temmelig robust, og enkelt å gjøre mange konverteringer.

### Eksportere data fra ao
Det viktigste her er å få eksportert 'Østkoordinat' og 'Nordkoordinat' riktig. De må ut i bredde- og lengdegrads desimaltall.  Du finnner det nede på profil-sida. Det skal stå "Geographic(WGS 84)" på Koordinatsystem.   
![](img/profil.png)
![](img/koordinatsystem.png)  
Så kan du gå inn på "Søk Funn" og velge en grei tidsperiode. Best å begynne med noe som bare gir noen få (<100) obeservasjoner. Eksporten startes med "Presentere funn" og "Eksportere data".   

Jeg anbefaler å ikke endre filnavnet på excel-fila, men kopiere det eksporterte xlsx-dokumentet rett over hit til mappa der denne notebooken ligger. Kanskje lager jeg et opplegg senere for å konvertere flere filer i slengen. Enn så lenge må man kjøre dette programmet en gang per regneark som skal konverteres. 





In [146]:
# !pip install pandas
# !pip install pyproj
# !pip install openpyxl
import pandas as pd
from pathlib import Path

### Legg inn navnet på fila i ruta under, mellom hermetegnene:

In [147]:

ao_source = "ExcelExport_8077412_Page_3.xlsx"

**I feltet under blir dataene fra ao lest innn med Pandas.**

In [149]:
# ebird_template = "ebird_record_format_template.csv"
ebird_path = Path(ao_source).stem+"-ebird.csv"
ao_df = pd.read_excel(ao_source, skiprows=2)

# for idx, value in ao_df.loc[1].iteritems():
#     print(idx, value)

ao_df.head(4)

,Id,Taksonsorteringsrekkefølge,Valideringsstatus,Rødlistekategori,Artsnavn,Vitenskapelig navn,Autor,Skjermet funn,Antall,Enhet,...,Samlingsnummer,Samlingsbeskrivelse,Artsbestemt av,Bestemmelsesdato,Bekrefter,Bekreftelsesdato,Redigeringsansvarlig,Rapportør,Observatører,Prosjekt
0,28521548,70413656,Ikke validert (funnet er ikke kvalitetssikret),NaN,fossekall,Cinclus cinclus,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Egil Rønningstad,Egil Rønningstad,"Monica Rønningstad, Egil Rønningstad",NaN
1,28521511,70410787,Ikke validert (funnet er ikke kvalitetssikret),NaN,blåstrupe,Luscinia svecica,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Egil Rønningstad,Egil Rønningstad,"Monica Rønningstad, Egil Rønningstad",NaN
2,28467966,70403698,Ikke validert (funnet er ikke kvalitetssikret),NaN,løvsanger,Phylloscopus trochilus,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Monica Rønningstad,Monica Rønningstad,"Egil Rønningstad, Monica Rønningstad",NaN
3,28467965,70402168,Ikke validert (funnet er ikke kvalitetssikret),NaN,låvesvale,Hirundo rustica,"Linnaeus, 1758",Nei,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Monica Rønningstad,Monica Rønningstad,"Egil Rønningstad, Monica Rønningstad",NaN


#### I feltet under utvider vi tabellen vi har lest inn, med feltete som tilhører ebird.

In [140]:
ebird_headers = ['Common Name',  'Genus', 'Species', 'Number', 'Species Comments', 'Location Name', 'Latitude', 'Longitude',
 'Date', 'Start Time', 'State/Province', 'Country Code', 'Protocol', 'Number of Observers', 'Duration', 'All observations reported?',
 'Effort Distance Miles', 'Effort area acres', 'Submission Comments']
# if not ebird_headers:
#     print("Reading ebird columns from template")
#     ebird_headers = list(pd.read_csv(ebird_template, sep=";").columns)
# # Add these headers to the ao list before we populate them
for ebird_header in ebird_headers:
    ao_df["ebird-"+ebird_header]= ""
# ebird_headers

Det under er noe som ikke brukes, men som jeg vil ta vare på til en annen gang.

In [141]:
# Tried tp use the Originale koordinater, but that is empty for personal locations. These are therefore not used.
# import pyproj
# from pyproj import CRS, Proj

# def digitsonly(text:str):
#     return int("".join([c for c in text if c.isdigit()]))
# def latlong_from_orig(orig_text:str):
#     splits = orig_text.split(" ")
#     easting = digitsonly(splits[0])
#     northing = digitsonly(splits[1])
#     zone = digitsonly(splits[3])
#     precision = digitsonly(splits[4])
#     p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
#     lon, lat = p(easting, northing, inverse=True)
#     return lat, lon

# latlong_from_orig("Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)")

## Konverteringen
Her går vi gjennom alle felter som vi trenger i ebird sine kolonner, og legger inn dit riktig informasjon fra excel-arket vi fikk fra AO.  
Jeg har tatt noen designvalg som man kan ønske annerledes. Da kan du gjøre endinger her, eller du kan ta csv-fila som blir produsert inn i et regneark igjen, og gjøre de siste endringene der. Hvis du vil legge inn noe annet på 'Protocol' eller 'All observations reported?' så pass på at du har lest og forstått hensikten med disse, og kravene de har til rapportering for andre valg her.

In [143]:

c = 'Genus' # Første del av latinske navnet
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"].str.split(" ").str.get(0) # Første del
c = 'Species' # ANdre del av latinske navnet
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"].str.split(" ").str.get(-1) # Siste del

c = 'Number'
ao_df["ebird-"+c] = ao_df["Antall"]

c =  'Date' # Dates must adhere to the following format: month/day/year (e.g., 12/27/2007).
ao_df["ebird-"+c] = ao_df["Startdato"].dt.strftime('%m/%d/%Y')

c = 'Location Name' # Lokalitetsnavn
ao_df["ebird-"+c] = ao_df["Lokalitetsnavn"]

c = 'Latitude'
ao_df["ebird-"+c] = ao_df["Nordkoordinat"]

c = 'Longitude'
ao_df["ebird-"+c] = ao_df["Østkoordinat"]

c = 'Country Code' # "NO" når "IKKE I NORGE" er tom. 
ao_df.loc[ao_df["IKKE I NORGE"].isnull(),"ebird-"+c] = "NO"

c = 'Protocol'
ao_df["ebird-"+c] = "Historical" #  or Incidental or Stationary https://support.ebird.org/en/support/solutions/articles/48000950859#anchorQuickProtocols

c = 'Number of Observers'
ao_df["ebird-"+c] = ao_df["Observatører"].str.split(",").str.len()

c =  'All observations reported?'
ao_df["ebird-"+c] = "N"

c = 'Submission Comments'
ao_df["ebird-"+c] = "Imported from file: "+ebird_path

In [144]:
ao_df[[c for c in ao_df.columns if c.startswith("ebird")]].head(4)

,ebird-Common Name,ebird-Genus,ebird-Species,ebird-Number,ebird-Species Comments,ebird-Location Name,ebird-Latitude,ebird-Longitude,ebird-Date,ebird-Start Time,ebird-State/Province,ebird-Country Code,ebird-Protocol,ebird-Number of Observers,ebird-Duration,ebird-All observations reported?,ebird-Effort Distance Miles,ebird-Effort area acres,ebird-Submission Comments
0,,Cinclus,cinclus,1,,Spranget,61.839439,9.732312,07/12/2014,,,NO,Historical,2,,N,,,Imported from file: ExcelExport_8077412_Page_1...
1,,Luscinia,svecica,1,,Krokåtbekkbue,61.854240,9.763304,07/13/2014,,,NO,Historical,2,,N,,,Imported from file: ExcelExport_8077412_Page_1...
2,,Phylloscopus,trochilus,1,,Rønningstad,60.873799,11.287466,05/05/2006,,,NO,Historical,2,,N,,,Imported from file: ExcelExport_8077412_Page_1...
3,,Hirundo,rustica,2,,Rønningstad,60.873799,11.287466,05/05/2006,,,NO,Historical,2,,N,,,Imported from file: ExcelExport_8077412_Page_1...


### Nå har vi flyttet informasjonen fra ao-feltene til ebird-feltene, og konvertert den ved behov
Nå skal vi bare kaste ut de kolonnene som hører til ao, og så skal vi lagre det vi har som csv. `index=False, header=None` er det fordi vi ikke må ha med den interne linjenummereringen i Pandas, og fordi kolonneoverskriftene ikke skal være der ved import i ebird.

In [145]:
ebird_df = ao_df[[c for c in ao_df.columns if c.startswith("ebird")]].copy() # Ligger i rett rekkefølge fra opprettelsen.
ebird_df.to_csv(ebird_path, index=False, header=None) # Ta bort kolonneoverskriftene.

Hvis alt har gått bra, så har du nå ei fil som slutter på `-ebird.csv`, og ellers heter det samme som excel-arket fra ao.
### Importere til ebird
Pass på å lese deg opp litt på formatet og hva kolonnene betyr. Kanskje jeg har gjort noen feil som gjør at du helst ikke ønsker å importere tusenvis av observasjoner før du har sjekket at ting blir rett. Husk at det er du, og ikke jeg som bruker dette programmet, når det er du som bruker det.  
#### Logg inn på e-bird og gå til submit.
https://ebird.org/submit  
![](img/import1.png)
#### Finn csv-fila og velg eBird record format
![](img/import2.png)
#### Gikk det bra?
Du får tilbakemelding om importen feilet.  
**Gode nyheter! Du kan gå tilbake og slette importen selv om den gikk gjennom.**
![](img/import3.png)
